# Welcome to the creation of models for ERP-based spellers tutorial!

Medusa is a framework designed for scientists and developers who investigate
novel signal processing algorithms, reducing the development and testing time
in real experiments. This includes not only the implementation of cutting-edge
signal processing methods, but also high level functionalities to assure the
persistence and reproducibility of the algorithms created within the framework.
One of they key features that makes Medusa so powerful is its ability to
implement and share standalone algorithms out of the box compatible with
Medusa applications.

In this notebook you will learn:
- How to create a custom model for ERP-based spellers
- Save the algorithm
- Use the algorithm in Medusa platform

Before this tutorial, make sure you have checked:
- [Algorithm creation tutorial](algorithms_tut_basic.ipynb)
- [Overview of erp_spellers module](erp_spellers_tut.ipynb)

Do not forget to check the documentation if you do not understand something!


## Introduction

In the world of brain-computer interfaces, novel algorithms arise every day to
improve these systems. However, most of these methods are not tested in online
experiments due to the technical complexity and time required to develop full
stack BCIs, putting in doubt their real usefulness. With Medusa, you can
focus on the development of new algorithms because, by following some simple
rules, you can implement a standalone algorithm to decode brain signals in
real time and put it in production within minutes! All of this, assuring
interoperability with existing frameworks such as sklearn, mne, etc. Ready?

## Imports

Import the modules that will be used in this notebook

In [1]:
# Built-in imports
import glob

# External imports
from sklearn.svm import SVC
import numpy as np
from tabulate import tabulate

# Medusa imports
from medusa import components
from medusa.meeg import meeg
from medusa.bci import erp_spellers


def print_acc_per_seq(acc_per_seq):
    table_cmd_acc_per_seq = ['Command decoding acc']
    cmd_acc_per_seq = np.char.mod('%.2f', acc_per_seq*100).astype(str).tolist()
    table_cmd_acc_per_seq += cmd_acc_per_seq
    headers = [''] + list(range(1, 16))
    print('\nTrain accuracy per number of sequences of stimulation:\n')
    print(tabulate([table_cmd_acc_per_seq], headers=headers))

## Download the dataset

As strong supporters of open science, we have released and adapted some
valuable datasets that can be very useful for researchers and practitioners.
These datasets can be downloaded manually from www.medusa.com/datasets/ or
using a simple API. In this case, we will use the API. Run the following cell
to download the GIB-UVa ERP dataset [1].

Each file is an instance of medusa.data_structures.Recording. This class
contains the information of the performed experiment, and the recorded
biosignals. In this case, the recordings contain an instance of
medusa.components.ERPSpellerData, which is the default class for
ERPBasedSpellers. Additionally, all recordings contain a medusa.meeg.EEG
instance.

In [2]:
# TODO: Download dataset
# dataset_folder = os.getcwd()

In [3]:
cha_set = meeg.EEGChannelSet()
cha_set.set_standard_montage(l_cha=['Fz', 'Cz', 'Pz', 'Oz'])
dataset = erp_spellers.ERPSpellerDataset(channel_set=cha_set,
                                         biosignal_att_key='eeg',
                                         experiment_att_key='erpspellerdata',
                                         experiment_mode='train')

print('OK!')

OK!


C:\Users\Diego\AppData\Local\Programs\Python\Python310\lib\site-packages\medusa\bci\erp_spellers.py:650: UserWarning: Parameter fs is None. The consistency of the dataset cannot be assured. Still, you can use target_fs parameter for feature extraction and everything should be fine.
  warnings.warn('Parameter fs is None. The consistency of the '


## Add recordings to the dataset

Now, we have to add the recordings to the dataset. With this purpose, we read
the files that were downloaded and use the function add_recordings of our
dataset. Note that this function admits instances of medusa.components.Recording
or a list of paths. For convenience, we will use the second option in this case.

In [4]:
folder = 'data'
file_pattern = '*.rcp.bson'
files = glob.glob('%s/%s' % (folder, file_pattern))
dataset.add_recordings(files)

print('OK!')

ValueError: Parameter recordings is empty!

## Algorithm

The next step is to instantiate the methods that will compose the algorithm,
but take into account that only methods that inherit from
medusa.components.ProcessingMethod can be added to the
Algorithm class. Medusa framework includes a wide variety of signal processing
methods ready to use. Nevertheless, function and class wrappers have also been
designed to assure full interoperability with external packages.

To show these functionalities, we will implement a custom algorithm based on a
support vector machine (SVM) using the sklearn package. The algorithm will
have the following stages:
1. **Preprocessing:** frequency filtering using an IIR filter with order=5 and
cutoff frequences in (0.5, 10) Hz and spatial filtering using common average
reference (CAR).
2. **Feature extraction:** EEG epochs from (0, 1000) ms after each stimulus
onset, baseline normalization (-250, 0) ms and downsampling to 20 Hz
3. **Feature classification:** SVM classifier using the implementation of
sklearn wrapped with ProcessingClassWrapper.
4. **Command decoding:** additional data processing to decode the selected
commands form predicted scores of EEG epochs.
4. **Model assessment:** method to calculate the accuracy of the model as a
function of the number of sequences of stimulation

Let's do it!

In [5]:
# 1. Preprocessing
prep = erp_spellers.StandardPreprocessing()

# 2. Feature extractor
feat_ext = erp_spellers.StandardFeatureExtraction()

# 3. Classifier. We must define the methods and output variables that will be
# exposed to the algorithm. In this case, we will need fit and predict_proba.
# See the sklearn documentation to learn more about this classifier.
clf = components.ProcessingClassWrapper(
    SVC(), fit=[], predict=['y_pred']
)

# 4. Command decoding function to decode the predicted command from epochs
# scores
cmd_decoding = components.ProcessingFuncWrapper(
    erp_spellers.decode_commands,
    outputs=['spell_result', 'spell_result_per_seq', 'scores']
)

# 5. Method to calculate the accuracy of the classifier per number of
# sequences of stimulation
model_assessment = components.ProcessingFuncWrapper(
    erp_spellers.command_decoding_accuracy_per_seq,
    outputs=['spell_acc_per_seq']
)

# Create algorithm instance and add the methods
alg = erp_spellers.ERPSpellerModel()
alg.add_method('prep', prep)
alg.add_method('feat-ext', feat_ext)
alg.add_method('clf', clf)
alg.add_method('cmd-decoding', cmd_decoding)
alg.add_method('assessment', model_assessment)

print('OK!')

TypeError: Can't instantiate abstract class ERPSpellerModel with abstract methods build, configure

## Define model pipelines

Once the methods have been added to the algorithm, it's time to define the
algorithm processing pipelines. Models based on ERPSpellerModel, which
inherits from components.Algorithm, have to implement 2 pipelines: one to fit
the algorithm from a dataset, and one to predict commands from EEG signal.

In [6]:
def fit_dataset_pipeline():
    pipe = components.Pipeline()
    uid_0 = pipe.input(['dataset'])
    uid_1 = pipe.add(
        method_func_key='prep:fit_transform_dataset',
        dataset=pipe.conn_to(uid_0, 'dataset')
    )
    uid_2 = pipe.add(
        method_func_key='feat-ext:transform_dataset',
        dataset=pipe.conn_to(uid_1, 'dataset'),
    )
    uid_3 = pipe.add(
        method_func_key='clf:fit',
        X=pipe.conn_to(uid_2, 'x'),
        y=pipe.conn_to(uid_2, 'x_info',
                       conn_exp=lambda x_info: x_info['erp_labels']
        )
    )
    uid_4 = pipe.add(
        method_func_key='clf:predict',
        X=pipe.conn_to(uid_2, 'x')
    )
    uid_5 = pipe.add(
        method_func_key='cmd-decoding:decode_commands',
        scores=pipe.conn_to(uid_4, 'y_pred'),
        paradigm_conf=pipe.conn_to(
            uid_2, 'x_info',
            conn_exp=lambda x_info: x_info['paradigm_conf']),
        run_idx=pipe.conn_to(
            uid_2, 'x_info',
            conn_exp=lambda x_info: x_info['run_idx']),
        trial_idx=pipe.conn_to(
            uid_2, 'x_info',
            conn_exp=lambda x_info: x_info['trial_idx']),
        matrix_idx=pipe.conn_to(
            uid_2, 'x_info',
            conn_exp=lambda x_info: x_info['matrix_idx']),
        level_idx=pipe.conn_to(
            uid_2, 'x_info',
            conn_exp=lambda x_info: x_info['level_idx']),
        unit_idx=pipe.conn_to(
            uid_2, 'x_info',
            conn_exp=lambda x_info: x_info['unit_idx']),
        sequence_idx=pipe.conn_to(
            uid_2, 'x_info',
            conn_exp=lambda x_info: x_info['sequence_idx']),
        group_idx=pipe.conn_to(
            uid_2, 'x_info',
            conn_exp=lambda x_info: x_info['group_idx']),
        batch_idx=pipe.conn_to(
            uid_2, 'x_info',
            conn_exp=lambda x_info: x_info['batch_idx']),
    )
    uid_6 = pipe.add(
        method_func_key='assessment:command_decoding_accuracy_per_seq',
        selected_commands_per_seq=pipe.conn_to(
            uid_5, 'spell_result_per_seq'
        ),
        target_commands=pipe.conn_to(
            uid_2, 'x_info',
            conn_exp=lambda x_info: x_info['spell_target']
        )
    )
    return pipe

def predict_pipeline():
    pipe = components.Pipeline()
    uid_0 = pipe.input(['times', 'signal', 'fs', 'x_info'])
    uid_1 = pipe.add(
        method_func_key='prep:fit_transform_signal',
        signal=pipe.conn_to(uid_0, 'signal'),
        fs=pipe.conn_to(uid_0, 'fs')
    )
    uid_2 = pipe.add(
        method_func_key='feat-ext:transform_signal',
        times=pipe.conn_to(uid_0, 'times'),
        signal=pipe.conn_to(uid_1, 'signal'),
        fs=pipe.conn_to(uid_0, 'fs'),
        onsets=pipe.conn_to(
            uid_0, 'x_info',
            conn_exp=lambda x_info: x_info['onsets']
        ),
    )
    uid_3 = pipe.add(
        method_func_key='clf:predict',
        X=pipe.conn_to(uid_2, 'x'),
    )
    uid_4 = pipe.add(
        method_func_key='cmd-decoding:decode_commands',
        scores=pipe.conn_to(uid_3, 'y_pred'),
        paradigm_conf=pipe.conn_to(
            uid_0, 'x_info',
            conn_exp=lambda x_info: x_info['paradigm_conf']),
        run_idx=pipe.conn_to(
            uid_0, 'x_info',
            conn_exp=lambda x_info: x_info['run_idx']),
        trial_idx=pipe.conn_to(
            uid_0, 'x_info',
            conn_exp=lambda x_info: x_info['trial_idx']),
        matrix_idx=pipe.conn_to(
            uid_0, 'x_info',
            conn_exp=lambda x_info: x_info['matrix_idx']),
        level_idx=pipe.conn_to(
            uid_0, 'x_info',
            conn_exp=lambda x_info: x_info['level_idx']),
        unit_idx=pipe.conn_to(
            uid_0, 'x_info',
            conn_exp=lambda x_info: x_info['unit_idx']),
        sequence_idx=pipe.conn_to(
            uid_0, 'x_info',
            conn_exp=lambda x_info: x_info['sequence_idx']),
        group_idx=pipe.conn_to(
            uid_0, 'x_info',
            conn_exp=lambda x_info: x_info['group_idx']),
        batch_idx=pipe.conn_to(
            uid_0, 'x_info',
            conn_exp=lambda x_info: x_info['batch_idx']),
    )
    return pipe

alg.add_pipeline('fit_dataset', fit_dataset_pipeline())
alg.add_pipeline('predict', predict_pipeline())

print('OK!')

NameError: name 'alg' is not defined

## Fit model

Time to fit the model! call to function fit to execute fit-dataset pipeline.

In [7]:
# Execute fit pipeline
fit_res = alg.fit_dataset(dataset)
print_acc_per_seq(fit_res[6]['res']['spell_acc_per_seq'])

NameError: name 'alg' is not defined

## Predict commands

Time to predict some commands simulating an online experiment! call function
predict to execute predict pipeline.

In [8]:
# Get some signal
rec = dataset.recordings[0]
times = rec.eeg.times
signal = rec.eeg.signal
fs = rec.eeg.fs
l_cha = rec.eeg.channel_set.l_cha
x_info = {'onsets': rec.erpspellerdata.onsets,
          'paradigm_conf': [rec.erpspellerdata.paradigm_conf],
          'run_idx': np.zeros_like(rec.erpspellerdata.onsets),
          'trial_idx': rec.erpspellerdata.trial_idx,
          'matrix_idx': rec.erpspellerdata.matrix_idx,
          'level_idx': rec.erpspellerdata.level_idx,
          'unit_idx': rec.erpspellerdata.unit_idx,
          'sequence_idx': rec.erpspellerdata.sequence_idx,
          'group_idx': rec.erpspellerdata.group_idx,
          'batch_idx': rec.erpspellerdata.batch_idx}

# Execute predict pipeline
predict_res = alg.predict(times, signal, fs, l_cha, x_info)

print('\nCommand decoding results:')
print(rec.erpspellerdata.spell_target)
print(predict_res[4]['res']['spell_result'])

IndexError: list index out of range

## Persistence

The Algorithm class includes persistence options to save the algorithm in
the current state. Medusa uses dill as serialization tool and thus it has
the same advantages and disadvantages of this tool.

It is possible to come across classes that are not directly serializable with
dill (e.g., keras models). In such cases, override methods 'to_pickleable_obj'
and 'from_pickleable_obj' of class Processing method.

Execute the next cell to save and load the previous model.

In [9]:
# Save algorithm
alg.save('alg.pkl')

# Load algorithm
loaded_alg = erp_spellers.ERPSpellerModel.load('alg.pkl')

# Predict with the loaded model
predict_res = loaded_alg.predict(times, signal, fs, l_cha, x_info)
print('\nCommand decoding results:')
print(rec.erpspellerdata.spell_target)
print(predict_res[4]['res']['spell_result'])

NameError: name 'alg' is not defined

## Standalone models

Congratulations! The file alg.pkl in your working directory contains a
standalone version of our algorithm. To load and use it in a different
script or machine, use the following code:

    >>> from medusa.bci import erp_spellers
    >>> alg = erp_spellers.ERPSpellerModel.load('alg.pkl')

Standalone algorithms are very useful for developers and scientists that design
add-hoc algorithms for a certain problem, database, etc, and want to share them
in an easy and quick way. Moreover, they are compatible with Medusa platform
apps.

Remember that only algorithms that contain methods accessible in the destination
machine can be distributed as a single file. For example, our example
can only be loaded in python environments which have sklearn installed. This
shouldn't be a problem, even for the most complex examples, due to the huge
amount of data processing packages available nowadays. Additionally, note
that dill is able to deserialize functions from scratch.

In the rare case that the available packages and dill functionalities don't suit
your needs, you have 2 options to distribute your algorithm: distribute your
code along with the algorithm file or create your own package in PyPI to easily
install your methods in any computer.

## Conclusion

That's all for now! Now you have a comprehensive picture on how to create and
use your own models for ERP-based spellers As you can see, you can build full
signal processing pipelines in a very flexible and easy way with few code
lines using Medusa!

See you in the next tutorial!